In [36]:
# Imports 
from datasets import load_dataset
import os
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm import tqdm
from transformers import WhisperProcessor, WhisperForConditionalGeneration, MarianMTModel, MarianTokenizer


In [37]:
# Load the europarl dataset training split
dataset = load_dataset("tj-solergibert/Europarl-ST", split="train")


In [38]:
# Filter the dataset where the speech is english, there is a french transcription, and the audio file is present 
dataset = dataset.filter(lambda example: example['original_language'] == 'en' and 'fr' in example['transcriptions'].keys() and example["transcriptions"]['fr'] is not None and os.path.exists(example["audio_path"]))
print(dataset)

# Select a subset of the filtered dataset for training
dataset = dataset.select(range(20))
print(dataset)

# Display some examples from the filtered dataset
for example in dataset.select(range(5)):
    print(example)

Dataset({
    features: ['original_speech', 'original_language', 'audio_path', 'segment_start', 'segment_end', 'transcriptions'],
    num_rows: 31777
})
Dataset({
    features: ['original_speech', 'original_language', 'audio_path', 'segment_start', 'segment_end', 'transcriptions'],
    num_rows: 20
})
{'original_speech': 'Mr President, I know that I will not be popular for making a long speech at this time, but my two fellow-rapporteurs, with whom I have worked very closely as a team, have made short statements so I want to keep the team spirit together.', 'original_language': 'en', 'audio_path': 'en/audios/en.20080924.23.3-123.m4a', 'segment_start': 0.0, 'segment_end': 14.470000267028809, 'transcriptions': {'de': 'Herr Präsident! Ich weiß, dass ich mir keine Freunde mache, wenn ich um diese Uhrzeit eine lange Rede halte, doch meine beiden Mitberichterstatter, mit denen ich sehr eng im Team zusammengearbeitet habe, haben kurze Stellungnahmen abgegeben, sodass ich den Teamgeist zusammen

In [39]:
# Load models and processors/tokenizers for whisper and marian
whisper_model_name = "openai/whisper-tiny"
translation_model_name = "Helsinki-NLP/opus-mt-en-fr"

whisper_processor = WhisperProcessor.from_pretrained(whisper_model_name)
print("Loaded whisper processor")
whisper_model = WhisperForConditionalGeneration.from_pretrained(whisper_model_name)
print("Loaded whisper conditional generator")
mt_tokenizer = MarianTokenizer.from_pretrained(translation_model_name)
print("Loaded mt tokenizer")
mt_model = MarianMTModel.from_pretrained(translation_model_name)
print("Loaded mt model")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded whisper processor
Loaded whisper conditional generator
Loaded mt tokenizer
Loaded mt model


In [40]:
# device config 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
whisper_model.to(device)
mt_model.to(device)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(59514, 512, padding_idx=59513)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(59514, 512, padding_idx=59513)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [41]:
import librosa 

audio_directory = "."

# preprocesses the dataset for training 
def preprocess(batch):
    # gets audio meta data
    audio_path = batch["audio_path"]
    segment_start = batch["segment_start"]
    segment_end = batch["segment_end"]
    
    # load the audio
    audio, sr = librosa.load(audio_path, sr=16000, offset=segment_start, duration=segment_end-segment_start)
    
    # process the audio using whisper
    inputs = whisper_processor(audio, sampling_rate=sr, return_tensors="pt")
    
    # pad the input to be length 3000 for whisper 
    input_features = inputs.input_features.squeeze(0)
    # if input_features.shape[-1] < 3000:
    #     padding = torch.zeros((input_features.shape[0], 3000 - input_features.shape[-1]))
    #     input_features = torch.cat([input_features, padding], dim=-1)
    
    batch["input_features"] = input_features
    
    # tokenize the transcription of the original speech for whisper labels 
    original_transcription = batch["original_speech"]
    tokenized_originals = whisper_processor.tokenizer(
        original_transcription,
        return_tensors="pt",
        padding="longest"
    ).input_ids.squeeze(0)
    
    batch["english_text"] = tokenized_originals
    
    french_transcription = batch["transcriptions"]["fr"]
    
    # Tokenize the target French text translation for marian
    tokenized_labels = mt_tokenizer(
        text_target=french_transcription,
        return_tensors="pt",
        padding="longest"
    ).input_ids.squeeze(0)
    
    batch["labels"] = tokenized_labels
    
    return batch

# Apply the preprocessing function to the dataset
dataset = dataset.map(preprocess, remove_columns=["audio_path", "original_speech", "original_language", "segment_start", "segment_end", "transcriptions"])


In [67]:
# collate function for tensor and padding before dataloader
def collate_fn(batch):
    input_features = [item['input_features'] for item in batch]
    labels = [item['labels'] for item in batch]
    english_text = [item['english_text'] for item in batch]
    
    # Convert all inputs to tensors if they are not already
    input_features = [torch.tensor(f) if not isinstance(f, torch.Tensor) else f for f in input_features]
    labels = [torch.tensor(l) if not isinstance(l, torch.Tensor) else l for l in labels]
    english_text = [torch.tensor(l) if not isinstance(l, torch.Tensor) else l for l in english_text]
    
    
    # pad sequences
    input_features = pad_sequence(input_features, batch_first=True)
    labels = pad_sequence(labels, batch_first=True, padding_value=mt_tokenizer.pad_token_id)
    english_text = pad_sequence(english_text, batch_first=True)
    
    
    return {
        'input_features': input_features,
        'labels': labels, 
        'english_text': english_text
    }

# create data loader using collate function
dataloader = DataLoader(dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)
print(dataloader)
# optimizers
whisper_optimizer = AdamW(whisper_model.parameters(), lr=5e-5)
mt_optimizer = AdamW(mt_model.parameters(), lr=5e-5)

# training loop
num_epochs = 3
output_dir = "./fine_tuned_models"

        
for epoch in range(3):
    whisper_model.train()
    mt_model.train()
    
    epoch_loss = 0.0
    for batch in tqdm(dataloader):
        
    
        input_features = (batch["input_features"])
        transcription = batch["english_text"]
        target_ids = batch["labels"]
    
        # forward pass through Whisper model with english transcription labels
        whisper_outputs = whisper_model(input_features, labels=transcription)
        
        predicted_ids = whisper_outputs.logits.argmax(dim=-1)
        
        # decode the predicted ids to text
        predicted_texts = whisper_processor.batch_decode(predicted_ids, skip_special_tokens=True)
        print(predicted_texts)

        # tokenize the decoded text for the translation model
        translation_inputs = mt_tokenizer(predicted_texts, return_tensors="pt", padding=True, truncation=True).input_ids.to(device)

        # forward pass through Translation model with Whisper's transcription as input
        translation_outputs = mt_model(input_ids=translation_inputs, labels=target_ids)

        # combine the losses for end-to-end training
        # combined_loss = whisper_outputs.loss + translation_outputs.loss
        # ombined_loss.backward()
       
        whisper_outputs.loss.backward()
        translation_outputs.loss.backward()
        
        whisper_optimizer.step()
        mt_optimizer.step()
        
        whisper_optimizer.zero_grad()
        mt_optimizer.zero_grad()

        epoch_loss += whisper_outputs.loss.item() + translation_outputs.loss.item()

    avg_loss = epoch_loss / len(dataloader)
    print(f"Epoch {epoch + 1}, Loss: {avg_loss}")

    # save the models
    whisper_model.save_pretrained(f"./fine_tuned_models/whisper_epoch_{epoch + 1}")
    whisper_processor.save_pretrained(f"./fine_tuned_models/whisper_epoch_{epoch + 1}")
    mt_model.save_pretrained(f"./fine_tuned_models/mt_epoch_{epoch + 1}")
    mt_tokenizer.save_pretrained(f"./fine_tuned_models/mt_epoch_{epoch + 1}")
    

  0%|          | 0/20 [00:00<?, ?it/s]

[' Presidentut,, you mentioned that word several times.']


  5%|▌         | 1/20 [00:02<00:56,  2.96s/it]

[' President, let us hope that the American proposals for purchases of toxic assets do work, because, if they do not, the contagion will almost certainly spread over here.']


 10%|█         | 2/20 [00:05<00:48,  2.72s/it]

...

 95%|█████████▌| 19/20 [00:43<00:02,  2.43s/it]

[' ask for your support to ensure that we have confident, well-informed consumers for electronic communications, who are also secure and know that their personal data is protected.']


100%|██████████| 20/20 [00:46<00:00,  2.31s/it]
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358, 50359, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}


Epoch 3, Loss: 0.706246136687696


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]], 'forced_eos_token_id': 0}


In [68]:
test_dataset = load_dataset("tj-solergibert/Europarl-ST", split="test")
test_dataset = test_dataset.filter(lambda example: example['original_language'] == 'en' and 'fr' in example['transcriptions'].keys() and example["transcriptions"]['fr'] is not None)
print(len(test_dataset))
print(test_dataset)
test_dataset = test_dataset.select(range(10))
test_dataset = test_dataset.map(preprocess, remove_columns=["audio_path", "original_speech", "original_language", "segment_start", "segment_end", "transcriptions"])

# Create DataLoader with custom collate_fn
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)


# Paths to the saved models
whisper_model_path = "./fine_tuned_models/whisper_epoch_3"  # Update with your final epoch
mt_model_path = "./fine_tuned_models/mt_epoch_3"  # Update with your final epoch

# Load the models
# whisper_processor = WhisperProcessor.from_pretrained(whisper_model_path)
# whisper_model = WhisperForConditionalGeneration.from_pretrained(whisper_model_path)
# mt_tokenizer = MarianTokenizer.from_pretrained(mt_model_path)
# mt_model = MarianMTModel.from_pretrained(mt_model_path)

whisper_processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
whisper_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
mt_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
mt_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-fr")


# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
whisper_model.to(device)
mt_model.to(device)

whisper_model.eval()
mt_model.eval()
    
epoch_loss = 0.0
for batch in tqdm(test_dataloader):
    input_features = batch["input_features"].to(device)

    original_speech = whisper_processor.batch_decode(batch["english_text"].to(device), skip_special_tokens=True)[0]
    print("Original Speech", original_speech)
    # Generate transcription using Whisper model
    with torch.no_grad():
        whisper_outputs = whisper_model.generate(
            input_features,
            num_beams=5, 
            repetition_penalty=1.2, 
            no_repeat_ngram_size=2, 
            temperature=0.7,  
            top_k=50,  
            top_p=0.95  
        )
        
        transcription = whisper_processor.batch_decode(whisper_outputs, skip_special_tokens=True)[0]

    print("Generated Transcription:", transcription)

    # Translate the transcription using the translation model
    tokenized_transcription = mt_tokenizer(transcription, return_tensors="pt", padding="longest", truncation=True)
    tokenized_transcription = tokenized_transcription.input_ids.to(device)

    with torch.no_grad():
        translated_tokens = mt_model.generate(tokenized_transcription)
        translation = mt_tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

    print("Translation to French:", translation)

avg_loss = epoch_loss / len(dataloader)
print(f"Loss: {avg_loss}")
    


Using the latest cached version of the dataset since tj-solergibert/Europarl-ST couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /Users/jacksonbrouwer/.cache/huggingface/datasets/tj-solergibert___europarl-st/default/0.0.0/9635c1dc32c9a45e70e4373cf82e0c2e8fbe12c3 (last modified on Wed Aug 14 20:26:41 2024).


1214
Dataset({
    features: ['original_speech', 'original_language', 'audio_path', 'segment_start', 'segment_end', 'transcriptions'],
    num_rows: 1214
})


Map:   0%|          | 0/10 [00:00<?, ? examples/s]/var/folders/ds/zb_p84s57dl5dvdn322z_sd80000gn/T/ipykernel_4686/2065403734.py:13: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(audio_path, sr=16000, offset=segment_start, duration=segment_end-segment_start)
/usr/local/lib/python3.12/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
Map: 100%|██████████| 10/10 [00:00<00:00, 14.74 examples/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /Helsinki-NLP/opus-mt-en-fr/resolve/main/model.safetensors (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x144106fc0>: Failed to resolve \'huggingface.co\' ([Errno 8] nodename nor servname provided, or not known)"))'), '(Request ID: 9aa9d87c-ba65-48f3-9884-ac863c24f870)')

In [ ]:
# Calculate BLEU score of the original model and the trained model 
import sacrebleu
import jiwer

bleu_scores = []
wer_scores = []

for batch in tqdm(test_dataloader):
    input_features = batch["input_features"].to(device)
    target_ids = batch["labels"].to(device)

    original_speech = whisper_processor.batch_decode(batch["english_text"].to(device), skip_special_tokens=True)[0]
    
    print("Original Speech", original_speech)
    # Generate transcription using Whisper model
    with torch.no_grad():
        whisper_outputs = whisper_model.generate(
            input_features,
            num_beams=5, 
            repetition_penalty=1.2, 
            no_repeat_ngram_size=2, 
            temperature=0.7,  
            top_k=50,  
            top_p=0.95  
        )
        
        transcription = whisper_processor.batch_decode(whisper_outputs, skip_special_tokens=True)[0]

    # print("Generated Transcription:", transcription)

    # Translate the transcription using the translation model
    tokenized_transcription = mt_tokenizer(transcription, return_tensors="pt", padding="longest", truncation=True)
    tokenized_transcription = tokenized_transcription.input_ids.to(device)

    with torch.no_grad():
        translated_tokens = mt_model.generate(tokenized_transcription)
        translation = mt_tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

    # print("Translation to French:", translation)

    # Calculate BLEU score
    reference = mt_tokenizer.decode(target_ids[0], skip_special_tokens=True)
    hypothesis = translation 
    bleu = sacrebleu.corpus_bleu([hypothesis], [[reference]])
    bleu_score = bleu.score
    bleu_scores.append(bleu_score)
    wer = jiwer.wer(reference, hypothesis)
    wer_scores.append(wer)

    print(f"Transcription: {transcription}")
    print(f"Generated Translation: {translation}")
    print(f"Reference Translation: {reference}")
    print(f"BLEU Score: {bleu_score}\n")
    print(f"WER: {wer}\n")

avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
avg_wer_score = sum(wer_scores) / len(wer_scores)

print(f"Average BLEU Score: {avg_bleu_score}")
print(f"Average WER: {avg_wer_score}")

  0%|          | 0/10 [00:00<?, ?it/s]

Original Speech Why have policymakers not learnt from previous crises, though stern warnings were sent?


 10%|█         | 1/10 [00:03<00:28,  3.12s/it]

Transcription:  Why haven't policy-making learned from previous crisis? Those turn-and-warnings were sent.
Generated Translation: Pourquoi l'élaboration des politiques n'a-t-elle pas appris de la crise précédente?
Reference Translation: Pourquoi les décideurs politiques n'ont-ils pas tiré les leçons des précédentes crises, en dépit des avertissements qui leur étaient adressés?
BLEU Score: 2.2008038097333027

Original Speech What about the flaws of the originate-and-distribute model, which has enhanced systemic risk?


 20%|██        | 2/10 [00:05<00:22,  2.76s/it]

...

Transcription:  The argument that regulations type of financial innovation I find ludicrous.
Generated Translation: L'argument selon lequel le type de réglementation d'innovation financière me paraît ridicule.
Reference Translation: L'argument selon lequel la réglementation étoufferait l'innovation financière est tout simplement grotesque.
BLEU Score: 13.67440667823257

Original Speech What about banks engaging in casino-type transactions?


 90%|█████████ | 9/10 [00:21<00:02,  2.12s/it]

Transcription:  What would banks engage in casino type transactions?
Generated Translation: Qu'est-ce que les banques effectueraient dans des transactions de type casino?
Reference Translation: Et les banques qui s'engagent dans des transactions de type casino?
BLEU Score: 57.067457770559976

Original Speech What about the ‘ shadow ’ banking sector, with its extreme leveraging and speculation?


100%|██████████| 10/10 [00:23<00:00,  2.32s/it]

Transcription:  What about the shadow banking sector with extreme leveraging and speculation?
Generated Translation: Qu'en est-il du secteur bancaire parallèle avec un effet de levier et des spéculations extrêmes?
Reference Translation: Et le secteur bancaire « caché », qui mise à l'extrême sur l'effet de levier et la spéculation?
BLEU Score: 9.330745616758765

Average BLEU Score: 21.901809259032778
